In [47]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import pandas as pd
import numpy as np
import pickle
import keras

In [32]:
data2=pd.read_csv(filepath_or_buffer='time_series/data/data2.csv',names=['value'])

In [22]:
#data2=data2.reset_index()

In [33]:
data2.head(10)

,value
0,1455.219971
1,1399.420044
2,1402.109985
3,1403.449951
4,1441.469971
5,1457.599976
6,1438.560059
7,1432.250000
8,1449.680054
9,1465.150024


In [34]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data2)

In [205]:
(data2.iloc[0].value-min(data2.value))/(max(data2.value)-min(data2.value))


(2175.030029, 676.530029)

In [206]:
(max(data2.value),min(data2.value))



(2175.030029, 676.530029)

In [40]:
train_length=int(len(scaled_data)*0.75)
train = scaled_data[0:train_length,:]
valid = scaled_data[train_length:,:]

In [41]:
train

array([[0.51964627],
       [0.48240909],
       [0.48420417],
       ...,
       [0.40638637],
       [0.42615948],
       [0.42606604]])

In [82]:
x_train, y_train = [], []

In [83]:
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])

In [84]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [87]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))


(3068, 60, 1)

In [88]:
x_train.shape


(3068, 60, 1)

In [51]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


In [52]:
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

Instructions for updating:
Use tf.cast instead.


Epoch 1/1


 - 115s - loss: 7.7381e-04


In [172]:
x_valid=[]
for i in range(len(valid)):
    x_valid.append(scaled_data[(i-60+train_length):(i+train_length),0])


(1043, 60)

In [173]:
x_valid=np.array(x_valid)


(1043, 60)

In [174]:
x_valid.shape


(1043, 60)

In [175]:
x_valid=np.reshape(x_valid, (x_valid.shape[0],x_valid.shape[1],1))

In [176]:
y_valid= model.predict(x_valid)

In [177]:
rms=np.sqrt(np.mean(np.power((valid-y_valid),2)))

In [178]:
rms

0.013552832189854768

In [225]:
#np.reshape(scaled_data[(i-60):(i),0],(1,60,1))


In [179]:
final={}
final['valid']=[i[0] for i in scaled_data]
final['prediction']=[None for i in range(train_length)]+[y[0] for y in y_valid]

In [180]:
final=pd.DataFrame(final)


In [193]:
#import matplotlib.pyplot as plt
#%matplotlib inline
#from matplotlib.pylab import rcParams
#rcParams['figure.figsize'] = 20,10

In [195]:
#plt.figure(figsize=(16,8))

#plt.plot(final[['valid','prediction']])
#plt.savefig('performance.png')

In [188]:

#saving model

with open('model.pkl','wb') as mp:
    pickle.dump(model,mp)

![Performance of LSTM](performance.png)

In [213]:
with open('scales.pkl','wb') as mp:
    scales=(max(data2.value),min(data2.value))
    pickle.dump(scales, mp)

In [215]:
with open('time_series/scales.pkl','rb') as mp:
    sc=pickle.load(mp)

In [210]:
import os
os.listdir()

['time_series', '.idea', 'venv']

In [216]:
sc

(2175.030029, 676.530029)

In [226]:
scaled_data[(i-60):(i),0]

array([0.26265597, 0.2603203 , 0.25905903, 0.26238902, 0.25690355,
       0.26210879, 0.25602271, 0.25526862, 0.26338334, 0.26533866,
       0.26126794, 0.26599932, 0.2669002 , 0.27537539, 0.27502837,
       0.27788449, 0.27993993, 0.27861863, 0.27985318, 0.28892222,
       0.28902902, 0.29055056, 0.28825489, 0.29742405, 0.29839174,
       0.30016678, 0.30389724, 0.29718382, 0.3007674 , 0.29675672,
       0.30296296, 0.303984  , 0.30917579, 0.30846846, 0.31437435,
       0.31191853, 0.31032367, 0.31954619, 0.31199201, 0.30160157,
       0.30535866, 0.30337002, 0.30612611, 0.30663997, 0.30029362,
       0.30167497, 0.31113112, 0.30916252, 0.31298633, 0.32114113,
       0.31737067, 0.31316652, 0.32062727, 0.31717712, 0.31400069,
       0.312032  , 0.30994992, 0.30868197, 0.31173842, 0.3125659 ])

In [ ]:
import requests
import json


In [311]:
inputs=list(data2.iloc[0:60].value.values)



In [390]:
r= requests.post(url='http://127.0.0.1:5051/data/',json={'load':inputs})

In [391]:
r.text

'{"y": "[[0.56127954]]"}'

In [367]:
os.listdir()

['data1_analysis.nb.html',
 'requirements.txt',
 'data2_lstm.ipynb',
 '.Rproj.user',
 'performance.png',
 'model.h5py',
 'model.pkl',
 '.git',
 'data',
 'README.md',
 'scales.pkl',
 'server.py',
 'data1_analysis.Rmd',
 'data2_analysis_files',
 'teste.save',
 'data2_analysis.Rmd',
 'data2_lstm.html',
 'Dockerfile',
 '.gitignore',
 'doc',
 'data2_analysis.html',
 'time_series.Rproj',
 'data1_analysis.html']

In [366]:

model.save(filepath='model.h5py')

In [291]:
class model_store():
    scales = (0, 0)
    model = None
    valid = False
    x = None

    def __init__(self):
        with open('scales.pkl', 'rb') as mp:
            output = pickle.load(mp)
            self.scales = output

        with open('model.pkl', 'rb') as mp:
            self.model = pickle.load(mp)

    def scale(self, x):
        return (x - self.scales[1]) / (self.scales[0] - self.scales[1])

    def input(self, x):
        self.valid = len(x) == 60
        if self.valid:
            scaled_data = self.scale(np.array(x))
            self.x = np.reshape(scaled_data, (1, 60, 1))

    def predict(self):
        if not self.valid:
            return False
        return self.model.predict(self.x)

In [304]:
import os
os.chdir('time_series')


FileNotFoundError: [Errno 2] No such file or directory: 'time_series'

In [336]:
x=r.json()['load']
modela=model_store()
modela.input(x)


In [338]:
modela.predict()

array([[0.56127954]], dtype=float32)

In [345]:
import hashlib

hashlib.md5(r.json()['load'].__str__().encode()).digest()

b'\x87\xfd\xdd\xd2\xe7\x90\x03\x8cz\x05+\xb5g\x19\x9e6'

In [361]:
import keras.models as km

In [362]:
kmm = km.load_model('teste.save')

In [365]:
kmm.predict(modela.x)

array([[0.56127954]], dtype=float32)

In [372]:
import keras

In [373]:
keras.__version__

'2.2.4'